# Basics about quantum simulation

In [3]:
using Yao, Yao.Blocks
using LinearAlgebra
using Test, BenchmarkTools

## Basic Register Operations

### Constructors

In [5]:
ψ1 = zero_state(5)
@assert nqubits(ψ1) == nactive(ψ1) == 5
@assert nremain(ψ1) == nremain(ψ1) == 0

The total number of qubits here is 5, they are all acitve by default. `active` qubits are also called `system` qubits that are visible to operations, remaining qubits are the `environment`.

In [6]:
join(product_state(3, 0b110), product_state(3, 0b001)) == product_state(6, 0b110001)

true

In [11]:
reg = product_state(5, 0b11100)
addbit!(copy(reg), 2) == product_state(7, 0b0011100)

true

In [12]:
ψ1 = rand_state(5)
ψ2 = rand_state(5)
@assert ψ1 |> isnormalized && ψ2 |> isnormalized
ψ3 = 0.3ψ1 + 2ψ2
@assert ψ3 |> isnormalized == false
@assert ψ3 |> normalize! |> isnormalized

### Adjoint

In [13]:
bra = rand_state(3)'

DefaultRegister{1, Complex{Float64}}
    active qubits: 3/3 [†]

In [14]:
@assert reg |> isnormalized
reg' * reg

1.0 + 0.0im

# Focus
To support modulized design of circuits, we use `focus!` to set some qubits as active, and the remaining are not supported!

In [15]:
reg = rand_state(10)
focus!(reg, 1:5)

DefaultRegister{1, Complex{Float64}}
    active qubits: 5/10

In [16]:
nactive(reg)

5

In [17]:
relax!(reg)

DefaultRegister{1, Complex{Float64}}
    active qubits: 10/10

In [18]:
focus!(reg, [3,1,5]) do r
    r.state = randn(8, 8) * r.state
    r
end

DefaultRegister{1, Complex{Float64}}
    active qubits: 10/10

# Measure

In [81]:
reg = rand_state(7)
measure(reg)

1-element Array{Int64,1}:
 49

In [92]:
reg = rand_state(7)
[measure!(reg) for i=1:5]

5-element Array{Array{Int64,1},1}:
 [36]
 [36]
 [36]
 [36]
 [36]

In [93]:
reg = rand_state(7)
[measure_reset!(reg, val=0b0111000) for i=1:5]

5-element Array{Array{Int64,1},1}:
 [1] 
 [56]
 [56]
 [56]
 [56]

In [97]:
reg = rand_state(7)
[measure_reset!(reg, 1:3, val=0b011) for i=1:5]

5-element Array{Array{Int64,1},1}:
 [7]
 [3]
 [3]
 [3]
 [3]

In [98]:
reg = rand_state(5)
reg3 = relax!(reg) ⊗ rand_state(4)

DefaultRegister{1, Complex{Float64}}
    active qubits: 9/9

In [99]:
select(reg3, 0b011100010) |> statevec

1-element Array{Complex{Float64},1}:
 -0.02163448025080393 - 0.04426622345562754im

In [102]:
select(repeat(reg3, 3) |> focus!(1,4,2), 0b110)

DefaultRegister{3, Complex{Float64}}
    active qubits: 0/6

In [9]:
reg1 = rand_state(3,3)
reg2 = rand_state(3,3)
join(reg2, reg1) 

DefaultRegister{3, Complex{Float64}}
    active qubits: 6/6

In [10]:
reg = rand_state(3)
addbit!(copy(reg), 3) ≈ join(zero_state(3), reg)

true

# Utilities

In [66]:
ψ1 = rand_state(6, 3)
ψ2 = rand_state(6, 3)
fidelity(ψ1, ψ2)

3-element Array{Float64,1}:
 0.2629200569833408  
 0.036133392527502646
 0.22117308688977222 

In [67]:
focus!(ψ1, [1,2,4])' * focus!(ψ2, [1,2,4])

3-element Array{Array{Complex{Float64},2},1}:
 [0.00583813+0.0326435im -0.0361655+0.018877im … -0.00402826-0.00535318im -0.0299721+0.0336662im; 0.00499055+0.0246059im -0.0206718-0.00923574im … -0.0145988+0.00785847im -0.021881-0.0211995im; … ; -0.0129979+0.0217014im 0.0113703-0.00925451im … 0.0144394+0.0359569im 0.0624196+0.0162656im; -0.0207398-0.0323418im 0.0397599+0.0196986im … 0.0290476-0.021112im 0.0167469+0.0182834im]    
 [0.0417005+0.000267743im -0.0473185-0.0562105im … 0.0253143+0.011994im -0.0363981+0.0131624im; 0.00761689+0.0388646im 0.0111785-0.0181938im … 0.0403567-0.0140694im -0.0460928-0.013771im; … ; -0.0306069-0.00211875im -0.0409247-0.00209542im … 0.00172109+0.0324756im -0.00669815+0.0147558im; 0.020308+0.0143948im 0.00457508-0.00698457im … 0.0118949+0.0462287im -0.00540798+0.00474091im] 
 [-0.0108797+0.0077124im -0.00965004+0.000541488im … -0.0540584-0.00296464im 0.00324668+0.00354565im; 0.0368419-0.0184345im -0.0233599+0.0453663im … -0.0453015-0.029949im -0.0567123+

# Batched Registers

In [108]:
@assert nbatch(ψ1) == 1
reg = rand_state(10, 8)
@assert nbatch(reg) == 8

In [110]:
repeat(reg, 3)   # repeat registers in batch dimension

DefaultRegister{24, Complex{Float64}}
    active qubits: 10/10

In [69]:
@. reg * 5 - 4 * reg ≈ reg

8-element BitArray{1}:
 true
 true
 true
 true
 true
 true
 true
 true

In [70]:
map(reg->reg |> put(5, 2=>X), repeat(ψ2, 3))

9-element Array{DefaultRegister{1,Complex{Float64},SubArray{Complex{Float64},2,Array{Complex{Float64},3},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Int64},true}},1}:
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6
 DefaultRegister{1, Complex{Float64}}
    active qubits: 3/6

# Blocks

In [36]:
X

X gate

In [37]:
mat(X)

2×2 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       1.0+0.0im
 1.0+0.0im     0     

In [38]:
px = put(3, 1=>X)

Total: 3, DataType: Complex{Float64}
put on (1)
└─ X gate


In [39]:
mat(px)

8×8 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       1.0+0.0im     0       …     0          0          0     
 1.0+0.0im     0          0             0          0          0     
    0          0          0             0          0          0     
    0          0       1.0+0.0im        0          0          0     
    0          0          0          1.0+0.0im     0          0     
    0          0          0       …     0          0          0     
    0          0          0             0          0       1.0+0.0im
    0          0          0             0       1.0+0.0im     0     

In [40]:
2X

Total: 1, DataType: Complex{Float64}
[2] X gate


In [41]:
mat(2X)

2×2 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       2.0+0.0im
 2.0+0.0im     0     

In [42]:
cx = control(3, 3, 1=>X)

Total: 3, DataType: Complex{Float64}
control(3)
└─ (1,)=>X gate


In [43]:
mat(cx)

8×8 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
 1.0+0.0im     0          0       …     0          0          0     
    0       1.0+0.0im     0             0          0          0     
    0          0       1.0+0.0im        0          0          0     
    0          0          0             0          0          0     
    0          0          0          1.0+0.0im     0          0     
    0          0          0       …     0          0          0     
    0          0          0             0          0       1.0+0.0im
    0          0          0             0       1.0+0.0im     0     